In [0]:
spark














































SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
flight_df= spark.read.format("csv")\
.option("header","true")\
.option("inferschema","true")\
.option("mode", "FAILFAST")\
.load("/FileStore/tables/2010_summary.csv")

flight_df.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [0]:
flight_df.printSchema()

root
 |-- DEST_COUNTRY_NAME: string (nullable = true)
 |-- ORIGIN_COUNTRY_NAME: string (nullable = true)
 |-- count: integer (nullable = true)



Define Schema

In [0]:
from pyspark.sql.types import StructField, StructType, StringType, IntegerType

In [0]:
my_schema=StructType(
[StructField("DEST_COUNTRY_NAME", StringType(), True),
StructField("ORIGIN_COUNTRY_NAME", StringType(), True),
StructField("count", IntegerType(), True)])


In [0]:
flight_df_schema = spark.read.format("csv")\
    .option("header", "false")\
    .option("skipRows",1)\
    .option("inferschema", "false")\
    .schema(my_schema)\
    .option("mode", "PERMISSIVE")\
    .load("/FileStore/tables/2010_summary.csv")

flight_df_schema.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



handling Corrupt data in csv

In [0]:
employee_df=spark.read.format("csv")\
.option("header", "true")\
.option("inferschema", "true")\
.option("mode","PERMISSIVE")\
.load("/FileStore/tables/employee-1.csv")
employee_df.show()

+---+--------+---+------+------------+--------+
| id|    name|age|salary|     address| nominee|
+---+--------+---+------+------------+--------+
|  1|  Manish| 26| 75000|       bihar|nominee1|
|  2|  Nikita| 23|100000|uttarpradesh|nominee2|
|  3|  Pritam| 22|150000|   Bangalore|   India|
|  4|Prantosh| 17|200000|     Kolkata|   India|
|  5|  Vikash| 31|300000|        null|nominee5|
+---+--------+---+------+------------+--------+



In [0]:
employee_df_1=spark.read.format("csv")\
.option("header", "true")\
.option("inferschema", "true")\
.option("mode","FAILFAST")\
.load("/FileStore/tables/employee-1.csv")
employee_df_1.show()

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-942344903635910>:6
      1 employee_df_1=spark.read.format("csv")\
      2 .option("header", "true")\
      3 .option("inferschema", "true")\
      4 .option("mode","FAILFAST")\
      5 .load("/FileStore/tables/employee-1.csv")
----> 6 employee_df_1.show()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:920, in DataFrame.show(self, n, truncate, vertical)
    914     raise PySparkTypeError(
    915         error_class="NOT_A_BOOLEAN",
    916        

In [0]:
employee_df_2=spark.read.format("csv")\
.option("header", "true")\
.option("inferschema", "true")\
.option("mode","DROPMALFORMED")\
.load("/FileStore/tables/employee-1.csv")
employee_df_2.show()

+---+------+---+------+------------+--------+
| id|  name|age|salary|     address| nominee|
+---+------+---+------+------------+--------+
|  1|Manish| 26| 75000|       bihar|nominee1|
|  2|Nikita| 23|100000|uttarpradesh|nominee2|
|  5|Vikash| 31|300000|        null|nominee5|
+---+------+---+------+------------+--------+



In [0]:
emp_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("salary", IntegerType(), True),
    StructField("address", StringType(), True),
    StructField("nominee", StringType(), True),
    StructField("_corrupt_record", StringType(), True)
])

In [0]:
employee_df_2 = spark.read.format("csv")\
    .option("header", "true")\
    .option("mode", "PERMISSIVE")\
    .schema(emp_schema)\
    .load("/FileStore/tables/employee-1.csv")

employee_df_2.show(truncate=False)

+---+--------+---+------+------------+--------+-------------------------------------------+
|id |name    |age|salary|address     |nominee |_corrupt_record                            |
+---+--------+---+------+------------+--------+-------------------------------------------+
|1  |Manish  |26 |75000 |bihar       |nominee1|null                                       |
|2  |Nikita  |23 |100000|uttarpradesh|nominee2|null                                       |
|3  |Pritam  |22 |150000|Bangalore   |India   |3,Pritam,22,150000,Bangalore,India,nominee3|
|4  |Prantosh|17 |200000|Kolkata     |India   |4,Prantosh,17,200000,Kolkata,India,nominee4|
|5  |Vikash  |31 |300000|null        |nominee5|null                                       |
+---+--------+---+------+------------+--------+-------------------------------------------+



In [0]:
employee_df_2 = spark.read.format("csv")\
    .option("header", "true")\
    .schema(emp_schema)\
    .option("badRecordsPath","/FileStore/tables/bad_recods")\
    .load("/FileStore/tables/employee-1.csv")

employee_df_2.show(truncate=False)

+---+------+---+------+------------+--------+---------------+
|id |name  |age|salary|address     |nominee |_corrupt_record|
+---+------+---+------+------------+--------+---------------+
|1  |Manish|26 |75000 |bihar       |nominee1|null           |
|2  |Nikita|23 |100000|uttarpradesh|nominee2|null           |
|5  |Vikash|31 |300000|null        |nominee5|null           |
+---+------+---+------+------------+--------+---------------+



In [0]:
%fs 
ls /FileStore/tables/bad_recods/20240717T164917/bad_records/

path,name,size,modificationTime
dbfs:/FileStore/tables/bad_recods/20240717T164917/bad_records/part-00000-665eab03-9884-468a-84a7-cf63a3e734e7,part-00000-665eab03-9884-468a-84a7-cf63a3e734e7,488,1721234959000


In [0]:
bad_data_df=spark.read.format("json").load("/FileStore/tables/bad_recods/20240717T164917/bad_records/")
bad_data_df.show()

+--------------------+--------------------+--------------------+
|                path|              reason|              record|
+--------------------+--------------------+--------------------+
|dbfs:/FileStore/t...|org.apache.spark....|3,Pritam,22,15000...|
|dbfs:/FileStore/t...|org.apache.spark....|4,Prantosh,17,200...|
+--------------------+--------------------+--------------------+



In [0]:
%fs 
ls /FileStore/tables/

path,name,size,modificationTime
dbfs:/FileStore/tables/2010_summary.csv,2010_summary.csv,7121,1721227624000
dbfs:/FileStore/tables/bad_recods/,bad_recods/,0,0
dbfs:/FileStore/tables/employee-1.csv,employee-1.csv,232,1721233030000


Load JSON

In [0]:
spark.read.format("json")\
.option("infer Schema", "true")\
.option("mode", "PERMISSIVE")\
.load("/FileStore/tables/line_delimited_json.json").show()

+---+--------+------+
|age|    name|salary|
+---+--------+------+
| 20|  Manish| 20000|
| 25|  Nikita| 21000|
| 16|  Pritam| 22000|
| 35|Prantosh| 25000|
| 67|  Vikash| 40000|
+---+--------+------+



In [0]:
# %fs
# rm /FileStore/tables/line_delimited_json.json

res6: Boolean = true

In [0]:
spark.read.format("json")\
.option("infer Schema", "true")\
.option("mode", "PERMISSIVE")\
.load("/FileStore/tables/line_delimited_json_extrafield.json").show()

+---+------+--------+------+
|age|gender|    name|salary|
+---+------+--------+------+
| 20|  null|  Manish| 20000|
| 25|  null|  Nikita| 21000|
| 16|  null|  Pritam| 22000|
| 35|  null|Prantosh| 25000|
| 67|     M|  Vikash| 40000|
+---+------+--------+------+



In [0]:
spark.read.format("json")\
.option("infer Schema", "true")\
.option("mode", "PERMISSIVE")\
.option("multiline", "true")\
.load("/FileStore/tables/Multi_line_correct.json").show()

+---+--------+------+
|age|    name|salary|
+---+--------+------+
| 20|  Manish| 20000|
| 25|  Nikita| 21000|
| 16|  Pritam| 22000|
| 35|Prantosh| 25000|
| 67|  Vikash| 40000|
+---+--------+------+



In [0]:
spark.read.format("json")\
.option("infer Schema", "true")\
.option("mode", "PERMISSIVE")\
.option("multiline", "true")\
.load("/FileStore/tables/Multi_line_incorrect.json").show()

+---+------+------+
|age|  name|salary|
+---+------+------+
| 20|Manish| 20000|
+---+------+------+



In [0]:
spark.read.format("json")\
.option("infer Schema", "true")\
.option("mode", "PERMISSIVE")\
.load("/FileStore/tables/corrupted_json.json").show()

+--------------------+----+--------+------+
|     _corrupt_record| age|    name|salary|
+--------------------+----+--------+------+
|                null|  20|  Manish| 20000|
|                null|  25|  Nikita| 21000|
|                null|  16|  Pritam| 22000|
|                null|  35|Prantosh| 25000|
|{"name":"Vikash",...|null|    null|  null|
+--------------------+----+--------+------+



In [0]:
spark.read.format("json")\
.option("infer Schema", "true")\
.option("mode", "PERMISSIVE")\
.option("multiline", "true")\
.load("/FileStore/tables/file5.json").printSchema()

root
 |-- code: long (nullable = true)
 |-- message: string (nullable = true)
 |-- restaurants: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- restaurant: struct (nullable = true)
 |    |    |    |-- R: struct (nullable = true)
 |    |    |    |    |-- res_id: long (nullable = true)
 |    |    |    |-- apikey: string (nullable = true)
 |    |    |    |-- average_cost_for_two: long (nullable = true)
 |    |    |    |-- cuisines: string (nullable = true)
 |    |    |    |-- currency: string (nullable = true)
 |    |    |    |-- deeplink: string (nullable = true)
 |    |    |    |-- establishment_types: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- events_url: string (nullable = true)
 |    |    |    |-- featured_image: string (nullable = true)
 |    |    |    |-- has_online_delivery: long (nullable = true)
 |    |    |    |-- has_table_booking: long (nullable = true)
 |    |    |    |-- i